## Import modules

In [1]:
#Data manipulation
import pandas as pd
import numpy as np
import random
import re
import random

#Data visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots as ps

import folium #Map
import streamlit as st #Web app
#from streamlit_jupyter import StreamlitPatcher, tqdm

#Configuration
import warnings
warnings.filterwarnings("ignore")

#Kaggle
import kaggle_data_download as kdd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Functions

In [2]:
#def random_hex_color():
#    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

## Download Data from Kaggle

In [3]:
kdd.download_dataset(dataset_name='sudalairajkumar/novel-corona-virus-2019-dataset', download_path='../Data')

Dataset URL: https://www.kaggle.com/datasets/sudalairajkumar/novel-corona-virus-2019-dataset
Dataset 'sudalairajkumar/novel-corona-virus-2019-dataset' downloaded and extracted to '../Data'


## Import data

In [4]:
main = pd.read_csv('../Data/covid_19_data.csv')

confirmed = pd.read_csv('../Data/time_series_covid_19_confirmed.csv')
recovered = pd.read_csv('../Data/time_series_covid_19_recovered.csv')
deaths = pd.read_csv('../Data/time_series_covid_19_deaths.csv')

us_confirmed = pd.read_csv('../Data/time_series_covid_19_confirmed_US.csv')
us_deaths = pd.read_csv('../Data/time_series_covid_19_deaths_US.csv')

## Data Manipulation

### Data Overview

In [5]:
main.head(2)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0


In [6]:
deaths.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0

In [7]:
us_deaths.head(2)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21
0,8

In [8]:
print('Checking data, seems to be in long format (main) and in wide format (remaining).\n'\
    'I will first merge the wide format data and see if there is additional info compared to main.\n'\
    'An importnt note is that there is no Lat, Long in main but has Last Update column (which corresponds to the recorded day of info).')

Checking data, seems to be in long format (main) and in wide format (remaining).
I will first merge the wide format data and see if there is additional info compared to main.
An importnt note is that there is no Lat, Long in main but has Last Update column (which corresponds to the recorded day of info).


### Merge data

In [9]:
main_places = (main['Province/State'] + ':' + main['Country/Region']).unique()
data_places = (us_deaths['Province_State'] + ':' + us_deaths['Country_Region']).unique()
#data_places[~pd.Series(data_places).isin(main_places)]
print(main_places[~pd.Series(main_places).isin(data_places)][random.sample(range(len(main_places) - 100), 10)], '\n')
print('Also, seems that the States between main and wide format data are the same but with different names (e.g. FL vs Florida, CA vs California)\n',
      'This will introduce problems while merging the main with the wide in order to keep Lat and Long info\n')

['Queensland:Australia' 'Tyva Republic:Russia'
 'Unassigned Location, WA:US' 'Sicilia:Italy' 'Puducherry:India'
 'Cesar:Colombia' 'Haryana:India' 'Falkland Islands (Islas Malvinas):UK'
 'Samara Oblast:Russia' 'Ulyanovsk Oblast:Russia'] 

Also, seems that the States between main and wide format data are the same but with different names (e.g. FL vs Florida, CA vs California)
 This will introduce problems while merging the main with the wide in order to keep Lat and Long info



In [10]:
#Check overlapped columns for wide format data
all_cols = sorted(set(us_confirmed.columns).union(set(us_deaths.columns)).union(set(deaths.columns)).union(set(confirmed.columns)).union(set(recovered.columns)))

columns_df = pd.DataFrame({
    'all_columns': all_cols,
    'confirmed': [col in confirmed.columns for col in all_cols],
    'deaths': [col in deaths.columns for col in all_cols],
    'recovered': [col in recovered.columns for col in all_cols],
    'us_confirmed': [col in us_confirmed.columns for col in all_cols],
    'us_deaths': [col in us_deaths.columns for col in all_cols],
})
columns_df[columns_df.iloc[:,1:].apply(lambda x: any(x != True), axis = 1)]

,all_columns,confirmed,deaths,recovered,us_confirmed,us_deaths
494,Admin2,False,False,False,True,True
495,Combined_Key,False,False,False,True,True
496,Country/Region,True,True,True,False,False
497,Country_Region,False,False,False,True,True
498,FIPS,False,False,False,True,True
500,Long,True,True,True,False,False
501,Long_,False,False,False,True,True
502,Population,False,False,False,False,True
503,Province/State,True,True,True,False,False
504,Province_State,False,False,False,True,True


In [11]:
#Remove unnecessary columns
us_confirmed = us_confirmed.iloc[:,~us_confirmed.columns.isin(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'])]
us_deaths = us_deaths.iloc[:,~us_deaths.columns.isin(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'])]

#Rename columns accordingly
cols = list(us_confirmed.columns)
cols[0:4] = list(confirmed.columns[0:4])
us_confirmed.columns = cols

cols = list(us_deaths.columns)
cols[0:4] = list(confirmed.columns[0:4])
us_deaths.columns = cols

In [12]:
# Merge the data by colnames
print(f'Wide format data have the same column names/order:\
 {list(recovered.columns) == list(confirmed.columns) == list(deaths.columns) == list(us_confirmed.columns) == list(us_deaths.columns)}\n')

confirmed['Status'], recovered['Status'], deaths['Status'], us_confirmed['Status'], us_deaths['Status'] = 'confirmed', 'recovered', 'death', 'confirmed', 'death'
data = pd.concat([confirmed, recovered, deaths, us_confirmed, us_deaths], axis = 0)

print(f'US data from non-US and US files seems to differ: {len(data[data['Country/Region'] == 'US']['Province/State'].unique()) == len(us_confirmed['Province/State'].unique())}.\n'\
    'Data in US files are seperated by State. Data in non-US files are overall aggregated.\n'\
    'I will remove overall aggregated !\n')

print(f'Merged data: {data[data['Country/Region'] == 'US']['Province/State'].isna().sum()},  Unmerged data: {us_deaths[us_deaths['Country/Region'] == 'US']['Province/State'].isna().sum()}\n',
      'There are 3 rows with missing State for US country which equals the number of non-US data (confirmed, recovered, deaths).\n',
      'These rows (except recovered cause there are not us_recovered file) will be removed !')
data = data[~((data['Country/Region'] == 'US') & (data['Province/State'].isna()) & (data['Status'].isin(['death', 'confirmed'])))]

Wide format data have the same column names/order: True

US data from non-US and US files seems to differ: False.
Data in US files are seperated by State. Data in non-US files are overall aggregated.
I will remove overall aggregated !

Merged data: 3,  Unmerged data: 0
 There are 3 rows with missing State for US country which equals the number of non-US data (confirmed, recovered, deaths).
 These rows (except recovered cause there are not us_recovered file) will be removed !


### Reshape merged data into long format

In [13]:
# Reshape data to long format
datalong = data.melt(id_vars=['Province/State','Country/Region', 'Lat', 'Long', 'Status'],
          value_vars=list(data.columns[data.columns.str.contains('\d', regex=True)]),
          var_name='Date', value_name='Cases')
print(f'Data reffered to about {len(datalong['Country/Region'].unique())} Countries/Regions. Seems reasonable value for the world data.')
print('Also, seems that the places between main and datalong are the same but with different names (e.g. FL vs Florida, CA vs California)')

Data reffered to about 193 Countries/Regions. Seems reasonable value for the world data.
Also, seems that the places between main and datalong are the same but with different names (e.g. FL vs Florida, CA vs California)


### Data cleaning

In [14]:
datalong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3702530 entries, 0 to 3702529
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Province/State  object 
 1   Country/Region  object 
 2   Lat             float64
 3   Long            float64
 4   Status          object 
 5   Date            object 
 6   Cases           int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 197.7+ MB


In [15]:
#Date format
datalong['Date'] = pd.to_datetime(datalong['Date'], format='%m/%d/%y', errors='coerce')
datalong.sort_values(by = ['Lat', 'Long', 'Status', 'Date'], inplace=True)

In [16]:
#Geographical info
datalong.loc[(datalong['Lat'] == 0) & (datalong['Long'] == 0),['Lat','Long']] = np.nan
datalong.loc[datalong['Province/State'] == 'Unknown', 'Province/State'] = np.nan

In [17]:
#Replace NA Lat-Long with the mean Lat-Long values of the corresponding country.
datalong['Country/Province'] =  datalong['Country/Region'] + ':' + datalong['Province/State']
datalong.loc[datalong['Country/Province'].isna(),'Country/Province'] = datalong.loc[datalong['Country/Province'].isna(),'Country/Region']
states_with_not_geo = (datalong.loc[datalong['Lat'].isna(),'Country/Province']).unique()

for state in states_with_not_geo:
    #print(state)
    condition1 = (datalong['Country/Province'] == state) & (datalong['Lat']).isna() & (datalong['Long'].isna())
    condition2 = (datalong['Country/Province'] == state) & (~datalong['Lat'].isna()) & (~datalong['Long'].isna())
    if condition2.sum() == 0:
        condition2 = (datalong['Country/Region'] == state.split(':')[0]) & (~datalong['Lat'].isna()) & (~datalong['Long'].isna())
        
    datalong.loc[condition1, ['Lat', 'Long']] = datalong.loc[condition2, ['Lat', 'Long']].mean().values
    #print(datalong.loc[condition1, ['Lat', 'Long']].head(2))
    #print(datalong.loc[condition2, ['Lat', 'Long']].mean().values)

In [18]:
#Remove duplicates
duplicates = datalong[datalong.duplicated(keep = False)]
print(f'Remove duplicate values.\n'\
 f'Total confirmed counted double: {duplicates[duplicates['Status'] == 'confirmed']['Cases'].sum()/2}\n'\
 f'Total recovered counted double: {duplicates[duplicates['Status'] == 'recovered']['Cases'].sum()/2}\n'\
 f'Total deaths counted double: {duplicates[duplicates['Status'] == 'death']['Cases'].sum()/2}')
datalong.drop_duplicates(inplace=True)

Remove duplicate values.
Total confirmed counted double: 211.0
Total recovered counted double: 0.0
Total deaths counted double: 131.0


In [19]:
# Aggregate data
datalong = datalong.groupby(['Country/Province', 'Status', 'Date'], dropna=False).agg({'Lat':'mean', 'Long':'mean', 'Cases': 'sum'}).reset_index()
#datalong[datalong['Country/Province'] == 'China:Henan'] # e.g.

In [20]:
#Format
#datalong['Lat'] = datalong['Lat'].astype('string')
#datalong['Long'] = datalong['Long'].astype('string')

In [21]:
#Clean recovered data
total_recovered = datalong[datalong['Status'] == 'recovered'].groupby(['Country/Province',]).agg({'Cases':'max'}).copy()

country = 'US'
for country in total_recovered.index:
    condition = (datalong['Country/Province'] == country) & (datalong['Status'] == 'recovered')
    idmax, indexmax = datalong.loc[condition,'Cases'].idxmax(), datalong.loc[condition,'Cases'].index.max()
    datalong.loc[idmax:indexmax,'Cases'] = total_recovered.loc[country, 'Cases']

### Missing Values

In [22]:
print(f'Missing Lat/Long values nelong to\
 {list(datalong.loc[(datalong['Lat']).isna() & (datalong['Long'].isna()),['Country/Province']]['Country/Province'].unique())} which are cruise ships.\n\
These values will be removed only from the map but considered in the overall analysis.')
datalong.isna().sum()

Missing Lat/Long values nelong to ['Diamond Princess', 'MS Zaandam'] which are cruise ships.
These values will be removed only from the map but considered in the overall analysis.


Country/Province       0
Status                 0
Date                   0
Lat                 2964
Long                2964
Cases                  0
dtype: int64

## Engineering

In [23]:
#Create active cases
datalong = datalong.pivot_table(index = ['Country/Province', 'Date',  'Lat', 'Long'], columns = 'Status', values = 'Cases').reset_index()
datalong['active'] = datalong['confirmed'] - datalong['recovered'] - datalong['death']
datalong = datalong.melt(id_vars=['Country/Province', 'Date', 'Lat', 'Long'],value_vars=['confirmed', 'recovered', 'death', 'active'], var_name='Status', value_name='Cases')

In [24]:
#Date columns
datalong['month'] = datalong['Date'].dt.month
datalong['month_name'] = datalong['Date'].dt.month_name()
datalong['day'] = datalong['Date'].dt.day_of_year
for year in datalong['Date'].dt.year.unique()[1:]:
    datalong.loc[datalong['Date'].dt.year == year, 'day'] = datalong.loc[datalong['Date'].dt.year == year, 'day'] + datalong['day'].max()
    datalong.loc[datalong['Date'].dt.year == year, 'month'] = datalong.loc[datalong['Date'].dt.year == year, 'month'] + datalong['month'].max()

In [25]:
#Cases column includes colulative values. I will create a new column with daily cases.
datalong['Daily_cases'] = datalong.groupby(['Country/Province', 'Status'])['Cases'].diff().fillna(0)
#There are some negative values in Daily_cases column which are errors.
#This could be due to the fact that some countries/regions have not reported cases for a while and then they report a big number of cases.
#This is not a problem for the overall analysis but it is a problem for the daily cases. 
datalong['Daily_cases_errors'] = (datalong['Daily_cases'] < 0)
datalong.loc[datalong['Daily_cases'] < 0, 'Daily_cases'] = 0

In [26]:
#Overall metrics
total_per_country = datalong.groupby(['Country/Province', 'Status'])['Cases'].max().reset_index().copy()
out = pd.DataFrame()
for country in total_per_country['Country/Province'].unique():
    d = total_per_country[total_per_country['Country/Province'] == country]
    if len(d['Status'].unique()) == 4:
        recovery_rate = (d[d['Status'] == 'recovered']['Cases'].values[0] / d[d['Status'] == 'confirmed']['Cases'].values[0])*100
        death_rate = (d[d['Status'] == 'death']['Cases'].values[0] / d[d['Status'] == 'confirmed']['Cases'].values[0])*100
    elif len(d['Status'].unique()) == 3:
        death_rate = (d[d['Status'] == 'death']['Cases'].values[0] / d[d['Status'] == 'confirmed']['Cases'].values[0])*100
        recovery_rate = np.nan
    else:
        recovery_rate, death_rate = np.nan, np.nan
    out = pd.concat([out, pd.DataFrame({'Country/Province': country,'Status': ['recovery_rate', 'death_rate'],'Cases': [recovery_rate, death_rate],})])
    
total_per_country = pd.concat([total_per_country, out], axis = 0).sort_values(by = ['Country/Province', 'Cases'])#.fillna(0)
total_per_country.dropna(inplace=True)
#total_per_country.loc[total_per_country['Status'] == 'death_rate', :].head(3)

In [27]:
total_per_country_wide = total_per_country.pivot(index = ['Country/Province'], columns = ['Status'], values = 'Cases').reset_index()
#total_per_country_wide['metric'] = (total_per_country_wide['death']/total_per_country_wide['death'].max())*((total_per_country_wide['confirmed']/total_per_country_wide['confirmed'].max()))
total_per_country_wide = total_per_country_wide.sort_values(by = 'death', ascending = False)
total_per_country_wide['Rank'] = range(1, len(total_per_country_wide) + 1)
total_per_country_wide['average_daily_cases'] = total_per_country_wide['confirmed']/(datalong['Date'].max() - datalong['Date'].min()).days
total_per_country_wide['average_daily_deaths'] = total_per_country_wide['death']/(datalong['Date'].max() - datalong['Date'].min()).days

In [28]:
#Monthly metrics
monthly_per_country = datalong.groupby(['Country/Province', 'Status', 'month', 'month_name'])['Cases'].max().reset_index().copy()
out = pd.DataFrame()
for country in monthly_per_country['Country/Province'].unique():
    for month in monthly_per_country['month'].unique():
        d = monthly_per_country[monthly_per_country['Country/Province'] == country]
        if len(d['Status'].unique()) == 3:
            recovery_rate = (d[(d['Status'] == 'recovered') & (d['month'] == month)]['Cases'].values[0] / d[(d['Status'] == 'confirmed') & ((d['month'] == month))]['Cases'].values[0])*100
            death_rate = (d[(d['Status'] == 'death') & ((d['month'] == month))]['Cases'].values[0] / d[(d['Status'] == 'confirmed') & ((d['month'] == month))]['Cases'].values[0])*100
        elif len(d['Status'].unique()) == 2:
            death_rate = (d[(d['Status'] == 'death') & ((d['month'] == month))]['Cases'].values[0] / d[(d['Status'] == 'confirmed') & ((d['month'] == month))]['Cases'].values[0])*100
            recovery_rate = np.nan
        else:
            recovery_rate, death_rate = np.nan, np.nan
        out = pd.concat([out, pd.DataFrame({'Country/Province': country, 'Status': ['recovery_rate', 'death_rate'],'month': month,'month_name': d['month_name'].values[0],'Cases': [recovery_rate, death_rate],})])

monthly_per_country = pd.concat([monthly_per_country, out], axis = 0).sort_values(by = ['Country/Province', 'Cases'])#.fillna(0)
monthly_per_country.dropna(inplace=True)
#monthly_per_country.loc[monthly_per_country['Status'] == 'death_rate', :].head(10)

In [29]:
total = total_per_country.groupby('Status').agg({'Cases': 'sum', 'Country/Province': 'nunique'}).reset_index().copy()
total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] = total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] / total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Country/Province"]
total

,Status,Cases,Country/Province
0,active,3.467603e+07,252
1,confirmed,1.702137e+08,331
2,death,3.533673e+06,331
3,death_rate,2.081470e+00,330
4,recovered,1.134889e+08,259
5,recovery_rate,8.719537e+01,257


## EDA

### Overall & By country - Metrics

In [30]:
choose = re.compile(r'Greece')
if choose.pattern == 'Global':
    total = total_per_country.groupby('Status').agg({'Cases': 'sum', 'Country/Province': 'nunique'}).reset_index().copy()
    total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] = total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] / total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Country/Province"]
    #Current week
    d = datalong[datalong['Date'] > (datalong['Date'].max() - pd.Timedelta(days = 6))].copy()
    d = d.groupby(['Status']).sum('Daily_cases').reset_index()
    current_confirmed, current_death, current_recovered, current_active = int(d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'death',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'active',['Daily_cases']].values[0])
    #Previous week
    week = 1
    dw = datalong[datalong['Date'].between(datalong['Date'].max() - pd.Timedelta(days = 2*(6*week + 1)), datalong['Date'].max() - pd.Timedelta(days = (6*week + 1)))].copy()
    dw = dw.groupby(['Status']).sum('Daily_cases').reset_index()
else:
    total = total_per_country[total_per_country['Country/Province'].str.contains(choose, regex=True)].groupby('Status').agg({'Cases': 'sum', 'Country/Province': 'nunique'}).reset_index().copy()
    total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] = total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Cases"] / total.loc[total['Status'].isin(['recovery_rate', 'death_rate']),"Country/Province"]
    #current week
    country = datalong[(datalong['Country/Province'].str.contains(choose, regex=True))].copy()
    d = country[country['Date'] > (country['Date'].max() - pd.Timedelta(days = 6))].copy()
    d = d.groupby(['Status']).sum('Daily_cases').reset_index()
    current_confirmed, current_death, current_recovered, current_active = int(d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'death',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0]), int(d.loc[d['Status'] == 'active',['Daily_cases']].values[0])
    #previous week
    week = 1
    dw = country[country['Date'].between(country['Date'].max() - pd.Timedelta(days = 2*(6*week + 1)), country['Date'].max() - pd.Timedelta(days = (6*week + 1)))].copy()
    dw = dw.groupby(['Status']).sum('Daily_cases').reset_index()

#COVID-19 Metrics: Choose
print(f'COVID-19 Metrics: {choose.pattern}\n'\
f'Time period data recorded: from {country['Date'].min().date()} to {country['Date'].max().date()}\n'\
f'Total Confirmed cases: {int(total.loc[total['Status'] == 'confirmed',['Cases']].sum()):,}\n'\
f'Total Deaths: {int(total.loc[total['Status'] == 'death',['Cases']].sum()):,}\n'\
f'Total Recovered* cases: {int(total.loc[total['Status'] == 'recovered',['Cases']].sum()):,}\n'\
f'Average Death rate: {float((total.loc[total['Status'] == 'death',['Cases']].values[0] / total.loc[total['Status'] == 'confirmed',['Cases']].values[0])*100):.2f}%\n'\
f'Average Recovery* rate: {float((total.loc[total['Status'] == 'recovered',['Cases']].values[0] / total.loc[total['Status'] == 'confirmed',['Cases']].values[0])*100):.2f}%\n'\
f'Total Active: {int(total.loc[total['Status'] == 'active',['Cases']].sum()):,}\n'\
)

#COVID-19 Metrics: Last 7 days (current week)
previous_confirmed, previous_death, previous_recovered, previous_active = int(dw.loc[dw['Status'] == 'confirmed',['Daily_cases']].values[0]), int(dw.loc[dw['Status'] == 'death',['Daily_cases']].values[0]), int(dw.loc[d['Status'] == 'recovered',['Daily_cases']].values[0]), int(dw.loc[dw['Status'] == 'active',['Daily_cases']].values[0])
val = 0 if current_active == 0 else (current_active - previous_active)/current_active
print('COVID-19 Metrics: Last 7 days (current week)\n'\
f'Time period data recorded: from {country['Date'].max().date() - pd.Timedelta(days = 6)} to {country['Date'].max().date()}\n'\
f'Total Confirmed cases (change compared to previous week): {current_confirmed:,} ({(current_confirmed-previous_confirmed)/previous_confirmed*100:.2f}%)\n'\
f'Total Deaths (change compared to previous week): {current_death:,} ({(current_death - previous_death)/current_death*100:.2f}%)\n'\
f'Total Recovered* cases: {current_recovered:,}\n'\
f'Total Active (change compared to previous week): {current_active:,} ({val*100:.2f}%)\n'\
#f'Average Death rate: {float((d.loc[d['Status'] == 'death',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n'\
#f'Average Recovery* rate: {float((d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n'\
)

#COVID-19 Metrics: From last 14 to 7 days (previous week)
print(f'COVID-19 Metrics: From last {2*(6*week + 1)} to {6*week + 1} days (previous week)\n'\
f'Time period data recorded: from {country['Date'].max().date() - pd.Timedelta(days = 2*(6*week + 1))} to {country['Date'].max().date() - pd.Timedelta(days = (6*week + 1))}\n'\
f'Total Confirmed cases: {previous_confirmed:,}\n'\
f'Total Deaths: {previous_death:,}\n'\
f'Total Recovered* cases: {previous_recovered:,}\n'\
f'Total Active: {previous_active:,}\n'\
#f'Average Death rate: {float((d.loc[d['Status'] == 'death',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n'\
#f'Average Recovery* rate: {float((d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n'\
)

#COVID-19 Metrics: Last 28 days (monthly)
d = country[country['Date'] > (country['Date'].max() - pd.Timedelta(days = 28))].copy()
d = d.groupby(['Status']).sum('Daily_cases').reset_index()
print('COVID-19 Metrics: Last 28 days (monthly)\n'\
f'Time period data recorded: from {country['Date'].max().date() - pd.Timedelta(days = 28)} to {country['Date'].max().date()}\n'\
f'Total Confirmed cases: {int(d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0]):,}\n'\
f'Total Deaths: {int(d.loc[d['Status'] == 'death',['Daily_cases']].values[0]):,}\n'\
f'Total Recovered* cases: {int(d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0]):,}\n'\
f'Average Death rate: {float((d.loc[d['Status'] == 'death',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n'\
f'Average Recovery* rate: {float((d.loc[d['Status'] == 'recovered',['Daily_cases']].values[0] / d.loc[d['Status'] == 'confirmed',['Daily_cases']].values[0])*100):.2f}%\n\n'\
f'Total Active: {int(d.loc[d['Status'] == 'active','Daily_cases'].sum()):,}\n'\
'*: Recovery cases (and also recovery rate) is underestimated as there where not adequete recovery data')

COVID-19 Metrics: Greece
Time period data recorded: from 2020-01-22 to 2021-05-29
Total Confirmed cases: 400,395
Total Deaths: 12,024
Total Recovered* cases: 93,764
Average Death rate: 3.00%
Average Recovery* rate: 23.42%
Total Active: 294,607

COVID-19 Metrics: Last 7 days (current week)
Time period data recorded: from 2021-05-23 to 2021-05-29
Total Confirmed cases (change compared to previous week): 10,591 (-29.62%)
Total Deaths (change compared to previous week): 252 (-63.49%)
Total Recovered* cases: 0
Total Active (change compared to previous week): 10,339 (-41.56%)

COVID-19 Metrics: From last 14 to 7 days (previous week)
Time period data recorded: from 2021-05-15 to 2021-05-22
Total Confirmed cases: 15,048
Total Deaths: 412
Total Recovered* cases: 0
Total Active: 14,636

COVID-19 Metrics: Last 28 days (monthly)
Time period data recorded: from 2021-05-01 to 2021-05-29
Total Confirmed cases: 53,973
Total Deaths: 1,571
Total Recovered* cases: 0
Average Death rate: 2.91%
Average Reco

### Overall & By country - Graphs

#### Graph1

In [31]:
choose = re.compile(r'US')
if choose.pattern == 'Global':
    #Set data
    confirmed_data = datalong[datalong['Status'] == 'confirmed'].copy()
    death_data = datalong[datalong['Status'] == 'death'][['Country/Province', 'Date', 'Cases','Daily_cases']].copy()
    death_data.columns = ['Country/Province', 'Date', 'Death','Daily_death']
    d = total_per_country_wide
else:
    #Set data
    confirmed_data = datalong[(datalong['Country/Province'].str.contains(choose, regex=True)) & (datalong['Status'] == 'confirmed')].copy()
    death_data = datalong[(datalong['Country/Province'].str.contains(choose, regex=True)) & (datalong['Status'] == 'death')][['Country/Province', 'Date', 'Cases','Daily_cases']].copy()
    death_data.columns = ['Country/Province', 'Date', 'Death','Daily_death']
    d = total_per_country_wide[total_per_country_wide['Country/Province'].str.contains(choose, regex=True)]

#Global total confirmed/death cases over time
global_daily = confirmed_data.groupby('Date').agg({'Cases': 'sum','Daily_cases': 'sum'}).reset_index()
death_daily = death_data.groupby('Date').agg({'Death': 'sum','Daily_death': 'sum'}).reset_index()
global_daily = global_daily.merge(death_daily, on = ['Date'], how = 'left')

#Create subplots
fig = ps(rows=2, cols=2, subplot_titles=[f'{choose.pattern} Cumulative Cases', f'{choose.pattern} Daily New Cases',
                                        f'{choose.pattern} Cumulative Deaths', f'{choose.pattern} Daily New Deaths'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]]
)
#Plot 1: Global cumulative cases
fig.add_trace(go.Scatter(x=global_daily['Date'], y=global_daily['Cases'],
    mode='lines',line=dict(color='#1f77b4', width=5),fill='tonexty',fillcolor='rgba(31,119,180,0.3)',name='',
    hovertemplate='Date: %{x}<br>Cases: %{y:,}<br>Deaths: %{customdata[0]:,}<extra></extra>',customdata=global_daily[['Death']].values
), row=1, col=1)
#Plot 2: Global daily new cases
fig.add_trace(go.Scatter(x=global_daily['Date'], y=global_daily['Daily_cases'],
    mode='lines',line=dict(color='#1f77b4', width=3),fill='tozeroy',fillcolor='rgba(31,119,180,0.3)', name='',
        hovertemplate='Date: %{x}<br>Cases: %{y:,}<br>Deaths: %{customdata[0]:,}<extra></extra>',customdata=global_daily[['Daily_death']].values
), row=1, col=2)
#Plot 3: Top countries cumulative
fig.add_trace(go.Scatter(x=global_daily['Date'], y=global_daily['Death'],
    mode='lines',line=dict(color='#d62728', width=5),fill='tonexty',fillcolor='rgba(214,39,40,0.3)',name='',
    hovertemplate='Date: %{x}<br>Cases: %{y:,}<br>Confirmed: %{customdata[0]:,}<extra></extra>',customdata=global_daily[['Cases']].values
), row=2, col=1)
#Plot 4: Top countries daily new cases
fig.add_trace(go.Scatter(x=global_daily['Date'], y=global_daily['Daily_death'],
    mode='lines',line=dict(color='#d62728', width=3),fill='tozeroy',fillcolor='rgba(214,39,40,0.3)', name='',
        hovertemplate='Date: %{x}<br>Cases: %{y:,}<br>Confirmed: %{customdata[0]:,}<extra></extra>',customdata=global_daily[['Daily_cases']].values
), row=2, col=2)
    
#Update layout
fig.update_layout(width = 1600, height=1000,title={'text': f'📊 {choose.pattern}: COVID-19 New Cases & Deaths Over Time','x': 0.5,'xanchor': 'center','font': {'size': 20, 'color': 'white'}},
showlegend=False, plot_bgcolor='rgb(30, 30, 30)',paper_bgcolor='rgb(30, 30, 30)', font=dict(color='white'))
#Update axes
fig.update_xaxes(showgrid=False, tickfont=dict(size=10, color='white'))
fig.update_yaxes(showgrid=True,gridcolor='rgb(100, 100, 100)',gridwidth=0.5,griddash='dash', tickfont=dict(size=15, color='white'))

#Add annotations
max_daily_case_date = global_daily.loc[global_daily['Daily_cases'].idxmax(), 'Date']
max_daily_case = global_daily['Daily_cases'].max()
max_daily_death_value = global_daily['Daily_death'].iloc[global_daily['Daily_cases'].idxmax()]
fig.add_annotation(x=max_daily_case_date,y=max_daily_case,text=f"Peak Daily Cases<br>Date: {max_daily_case_date.strftime('%Y-%m-%d')}<br>Confirmed: {max_daily_case:,.0f}<br>Deaths: {max_daily_death_value:,.0f}",
showarrow=False,arrowhead=1,arrowsize=1,arrowwidth=3,arrowcolor="white",ax=20,ay=-30,bgcolor="#1f77b4",bordercolor="white",borderwidth=2,font=dict(color='white'),row=1, col=2)

max_daily_death_date = global_daily.loc[global_daily['Daily_death'].idxmax(), 'Date']
max_daily_death = global_daily['Daily_death'].max()
max_daily_case_value = global_daily['Daily_cases'].iloc[global_daily['Daily_death'].idxmax()]
fig.add_annotation(x=max_daily_death_date,y=max_daily_death,text=f"Peak Daily Deaths<br>Date: {max_daily_death_date.strftime('%Y-%m-%d')}<br>Confirmed: {max_daily_death:,.0f}<br>Deaths: {max_daily_case_value:,.0f}",
showarrow=False,arrowhead=1,arrowsize=1,arrowwidth=3,arrowcolor="white",ax=20,ay=-30,bgcolor="#d62728",bordercolor="white",borderwidth=2,font=dict(color='white'),row=2, col=2)
#Print
fig.show()

#Print summary statistics
print("📈 Confirmed Cases Summary:\n"\
f"• Global Rank based on death rate (considering confirmed cases): {int(d['Rank'].min())}\n"\
f"• Average daily new cases: {global_daily['Daily_cases'].mean():,.0f}\n"
f"• Average daily new deaths: {global_daily['Daily_death'].mean():,.0f}\n"
f"• Death rate: {d['death_rate'].mean():,.2f}%\n",
#f"• Peak daily new cases: {max_daily_case:,} on {max_daily_case_date.strftime('%Y-%m-%d')}\n"\
#f"• Peak daily deaths: {max_daily_death:,} on {max_daily_death_date.strftime('%Y-%m-%d')}\n"\
)

📈 Confirmed Cases Summary:
• Global Rank based on death rate (considering confirmed cases): 15
• Average daily new cases: 54,700
• Average daily new deaths: 1,001
• Death rate: 1.63%



#### Graph 2

In [ ]:
#Set data
n = 5
# Get top n countries based on metric
top_n = total_per_country_wide.head(n).copy()

#Top n countries/states by total confirmed cases
total_per_country_wide['incidence_death'] = (total_per_country_wide['active']*total_per_country_wide['death_rate']/100).fillna(0).astype(int)
top_remained = total_per_country_wide.sort_values(by = 'active', ascending = False).head(n)

#Create subplots
fig = ps(rows=2, cols=2, subplot_titles=[f'📈 Top {n} countries based on deaths', f'📈 Top {n} countries based on active cases',
                                         f'📈 Death rate of top {n} countries<br>based on active cases',f'📈 Recovery Rate of top {n} countries<br>based on active cases'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]]
)

#Plot 1
fig.add_trace(go.Bar(x=top_n['Country/Province'],y=top_n['death'],orientation='v',
    marker=dict(color='#d62728', line=dict(color='white', width=2)),
    text=[f"{val:,.0f}" for val in top_n['death']], 
    textposition='outside',textfont=dict(size=12, color='white'),
    hovertemplate='<b>%{x}</b><br>' + 'Deaths: %{customdata[0]:,}<br>' + 'Cases: %{customdata[1]:,}<br>' +
    'Recovered: %{customdata[2]:,}<br>' + 'Death Rate: %{customdata[3]:.2f}%<br>' + 'Recovery Rate: %{customdata[4]:.2f}%<br><extra></extra>',
    customdata=np.column_stack((top_n['death'], top_n['confirmed'], top_n['recovered'], top_n['death_rate'],top_n['recovery_rate']))
), row = 1, col = 1)
#Plot 2
fig.add_trace(go.Bar(x=top_remained['Country/Province'],y=top_remained['active'],orientation='v',name = '',
    marker=dict(color='#ff7f0e', line=dict(color='white', width=2)),
    text=[f"{val:,.0f}" for val in top_remained['active']], 
    textposition='outside',textfont=dict(size=12, color='white'),
    hovertemplate='<b>%{x}</b><br>' + 'Active Cases: %{customdata[0]:,}<br>' + 'Death Rate: %{customdata[1]:.2f}%<br>' +
    'Recovery Rate: %{customdata[2]:.2f}%<br>' + 'Daily Cases: %{customdata[3]:.0f}<br>' + 'Daily Deaths: %{customdata[4]:.0f}<br><extra></extra>',
    customdata=np.column_stack((top_remained['active'],top_remained['death_rate'],top_remained['recovery_rate'], top_remained['average_daily_cases'], top_remained['average_daily_deaths']))
), row = 1, col = 2)
#Plot 3
fig.add_trace(go.Bar(x=top_remained['Country/Province'],y=top_remained['death_rate'],orientation='v',
    marker=dict(color='#d62728', line=dict(color='white', width=2)),
    text=[f"{val:.1f}%" for val in top_remained['death_rate']], 
    textposition='outside',textfont=dict(size=12, color='white'),
    hovertemplate='<b>%{x}</b><br>' + 'Death Rate: %{customdata[1]:.2f}%<br>' + 'Active Cases: %{customdata[0]:,}<br>' +
    'Recovery Rate: %{customdata[2]:.2f}%<br>' + 'Daily Cases: %{customdata[3]:.0f}<br>' + 'Daily Deaths: %{customdata[4]:.0f}<br><extra></extra>',
    customdata=np.column_stack((top_remained['active'],top_remained['death_rate'],top_remained['recovery_rate'], top_remained['average_daily_cases'], top_remained['average_daily_deaths']))
), row = 2, col = 1)
#Plot 4
fig.add_trace(go.Bar(x=top_remained['Country/Province'],y=top_remained['recovery_rate'],orientation='v',name = '',
    marker=dict(color='#2ca02c', line=dict(color='white', width=2)),
    text=[f"{val:.1f}%" for val in top_remained['recovery_rate']], 
    textposition='outside',textfont=dict(size=12, color='white'),
    hovertemplate='<b>%{x}</b><br>' + 'Recovery Rate: %{customdata[2]:.2f}%<br>' + 'Active Cases: %{customdata[0]:,}<br>' +
    'Death Rate: %{customdata[1]:.2f}%<br>' + 'Daily Cases: %{customdata[3]:.0f}<br>' + 'Daily Deaths: %{customdata[4]:.0f}<br><extra></extra>',
    customdata=np.column_stack((top_remained['active'],top_remained['death_rate'],top_remained['recovery_rate'], top_remained['average_daily_cases'], top_remained['average_daily_deaths']))
), row = 2, col = 2)

# Update layout
fig.update_layout(height=1000,width=1600,showlegend=False,plot_bgcolor='rgb(30, 30, 30)',paper_bgcolor='rgb(30, 30, 30)', font=dict(color='white'))
fig.update_xaxes(showgrid=False, tickfont=dict(size=12, color='white'), tickangle=45)
fig.update_yaxes(showgrid=True,gridcolor='rgb(100, 100, 100)',gridwidth=0.5,griddash='dash', tickfont=dict(size=15, color='white'))
fig.show()

# Print detailed ranking
print("=" * 80)
print(f"{'Rank':<4} {'Country/Province':<25} {'Confirmed':<12} {'Deaths':<10} {'Recovered':<12} {'Active':<12} {'Death Rate':<12} {'Recovery Rate'}")
print("-" * 80)

for idx, row in top_n.iterrows():
    print(f"{row['Rank']:<4} {row['Country/Province']:<25} {row['confirmed']:<12,.0f} {row['death']:<10,.0f} {row['recovered']:<12,.0f} {row['active']:<12,.0f} {row['death_rate']:<12.2f}% {row['recovery_rate']:.2f}%")

Rank Country/Province          Confirmed    Deaths     Recovered    Active       Death Rate   Recovery Rate
--------------------------------------------------------------------------------
1    Brazil                    16,471,600   461,057    14,496,224   1,548,092    2.80        % 88.01%
2    India                     27,894,800   325,972    25,454,320   3,745,237    1.17        % 91.25%
3    Mexico                    2,411,503    223,455    1,924,865    300,329      9.27        % 79.82%
4    United Kingdom            4,480,945    127,775    344          4,352,826    2.85        % 0.01%
5    Italy                     4,213,055    126,002    3,845,087    805,947      2.99        % 91.27%


#### Map

This section creates an interactive dark-themed world map showing COVID-19 data with:
- **Blue circles** for confirmed cases
- **Red circles** for deaths
- **Green circles** for recovered cases
- **Orange circles** for remaining cases (confirmed - deaths - recovered)
- Circle size proportional to case numbers
- Interactive date slider to filter data by time period
- Toggle buttons to show/hide different case types

In [33]:
map_data = datalong.copy()
map_data['Cases'] = map_data['Daily_cases'].copy()
def calculate_remaining_cases(df):
    pivot_data = df.pivot_table(
        index=['Country/Province', 'Date', 'Lat', 'Long'], columns='Status', values='Cases', fill_value=0).reset_index()
    pivot_data['remaining'] = pivot_data.get('active', 0)
    pivot_data['remaining'] = pivot_data['remaining'].clip(lower=0)
    
    return pivot_data

remaining_data = calculate_remaining_cases(map_data)

In [34]:
def create_animated_covid_map():
    """Create an animated map with Plotly's built-in slider"""
    date_range = pd.date_range(remaining_data['Date'].min(), remaining_data['Date'].max(), freq='M')    

    frames = []
    for i, date in enumerate(date_range):
        frame_data = remaining_data[remaining_data['Date'] <= date].copy()
        
        if frame_data.empty:
            continue
        latest_data = frame_data.groupby('Country/Province').last().reset_index()
        
        frame_traces = []
        case_config = {'confirmed': {'color': '#1f77b4', 'name': 'Confirmed'},'death': {'color': '#d62728', 'name': 'Deaths'},
                       'recovered': {'color': '#2ca02c', 'name': 'Recovered'},'active': {'color': '#ff7f0e', 'name': 'Active'}}
        
        for case_type, config in case_config.items():
            plot_data = latest_data[latest_data[case_type] > 0].copy()
            
            if plot_data.empty: continue
            
            max_cases = latest_data[case_type].max()
            min_cases = latest_data[case_type][latest_data[case_type] > 0].min()
            if max_cases > min_cases:
                normalized_size = ((np.log10(plot_data[case_type] + 1) - np.log10(min_cases + 1)) / (np.log10(max_cases + 1) - np.log10(min_cases + 1)) * 25 + 5)
            else:
                normalized_size = np.full(len(plot_data), 10)
            
            hover_text = [f"<b>{country}</b><br>" + f"Confirmed: {confirmed:,}<br>" + f"Deaths: {deaths:,}<br>" + f"Recovered: {recovered:,}<br>" + f"Remaining: {remaining:,}<br>"
                for country, confirmed, deaths, recovered, remaining in zip(plot_data['Country/Province'],plot_data['confirmed'],plot_data['death'],plot_data['recovered'],plot_data['active'])]
            frame_traces.append(go.Scattergeo(lon=plot_data['Long'],lat=plot_data['Lat'],text=hover_text,mode='markers',
                marker=dict(size=normalized_size,color=config['color'],opacity=0.6,sizemode='diameter',line=dict(width=1, color='white')),
                name=config['name'],hovertemplate='%{text}<extra></extra>',legendgroup=case_type,showlegend=True,visible=True))
        
        frames.append(go.Frame(data=frame_traces,name=date.strftime('%Y-%m-%d'),layout=go.Layout(title_text=f"COVID-19 Global Status - {date.strftime('%B %d, %Y')}")))
    
    fig = go.Figure(data=frames[0].data if frames else [],frames=frames)
    
    fig.update_layout(title={'text': 'Interactive COVID-19 World Map with Time Animation','x': 0.5,'font': {'size': 24, 'color': 'white'}},
        geo=dict(projection_type='natural earth',showland=True,landcolor='rgb(50, 50, 50)',coastlinecolor='rgb(100, 100, 100)',showocean=True,oceancolor='rgb(20, 20, 20)',showlakes=True,
                 lakecolor='rgb(20, 20, 20)',showcountries=True,countrycolor='rgb(100, 100, 100)',bgcolor='rgb(30, 30, 30)'),
        paper_bgcolor='rgb(30, 30, 30)',plot_bgcolor='rgb(30, 30, 30)',font=dict(color='white'),
        legend=dict(bgcolor='rgba(30, 30, 30, 0.8)',bordercolor='rgb(100, 100, 100)',borderwidth=1,font=dict(color='white'),x=0.02,y=0.98),width=1600,height=1000,
        updatemenus=[{'type': 'buttons','showactive': False,'y': 0.2,'x': 0.05,'xanchor': 'left','yanchor': 'bottom','pad': {'t': 10, 'r': 10},
            'buttons': [{'label': 'Play','method': 'animate','args': [None, {'frame': {'duration': 500, 'redraw': True},'fromcurrent': True,'transition': {'duration': 200}}]},
                {'label': 'Pause','method': 'animate','args': [[None], {'frame': {'duration': 0, 'redraw': False},'mode': 'immediate','transition': {'duration': 0}}]}]}],
        sliders=[{'active': 0,'yanchor': 'bottom','xanchor': 'center',
            'currentvalue': {'font': {'size': 16, 'color': 'white'},'prefix': 'Date: ','visible': True,'xanchor': 'center'},'pad': {'b': 10, 't': 10},'len': 0.9,'x': 0.5,'y': 0.05,
            'steps': [{'args': [[frame.name], {'frame': {'duration': 300, 'redraw': True},'mode': 'immediate','transition': {'duration': 200}}],
                       'label': pd.to_datetime(frame.name).strftime('%b %Y'),'method': 'animate'} for frame in frames]}])
    
    return fig

# Create and display the animated map
print("Creating animated COVID-19 map...")
animated_map = create_animated_covid_map()
animated_map.show()

print("🎬 Animated Map Features:\n"\
"✅ Click 'Play' to start animation through time\n"
"✅ Use the slider to navigate month by month\n"\
"✅ Click legend items to show/hide case types\n"\
"✅ Hover over circles for detailed information\n"\
"✅ Zoom and pan to explore regions\n")
print("🎨 Color Legend:\n"\
"🔵 Blue = Confirmed Cases\n"\
"🔴 Red = Deaths\n"\
"🟢 Green = Recovered Cases\n"\
"🟠 Orange = Remaining Active Cases")

Creating animated COVID-19 map...


🎬 Animated Map Features:
✅ Click 'Play' to start animation through time
✅ Use the slider to navigate month by month
✅ Click legend items to show/hide case types
✅ Hover over circles for detailed information
✅ Zoom and pan to explore regions

🎨 Color Legend:
🔵 Blue = Confirmed Cases
🔴 Red = Deaths
🟢 Green = Recovered Cases
🟠 Orange = Remaining Active Cases


## Output for Stramlit App

In [35]:
import os
os.makedirs('../Output', exist_ok=True)

datalong.to_csv('../Output/datalong.csv', index=False)
total_per_country.to_csv('../Output/total_per_country.csv', index=False)
total_per_country_wide.to_csv('../Output/total_per_country_wide.csv', index=False)